# Day 16: Packet Decoder


In [2]:
import math

In [88]:
def parse_data(source):
    try:
        with open(source) as f:
            source = f.read().strip()
    except:
        print("file not found, please try again")
        return -1
    return source

In [20]:
def evaluate_literal(bin_str):
    litvalue = ''
    for x in range(0, len(bin_str), 5):
        prefix = int(bin_str[x])
        litvalue += bin_str[x+1:x+5]
        if prefix == 0:
            break
    return [int(litvalue, 2), x+5]

In [21]:
def get_binary(hex_str):
    dec = int("1" + hex_str, 16)  # convert to decimal 
    #prepend 1 to the beginning of the hex string to ensure leading zeros are retained
    binary = bin(dec) # use bin to convert to binary
    return binary[3:]

In [99]:
def noSigBitsRemaining(stream, i):
    if stream[i:] == '':
        return True
    return int(stream[i:],2) == 0

In [156]:
def evaluate_operator(typeID, literals):
    if typeID == 0:
        return sum(literals), "Sum"
    elif typeID == 1:
        product = 1
        for x in literals:
            product = product * x
        return product, "Product"
    elif typeID == 2:
        return min(literals), "Minimum"
    elif typeID == 3:
        return max(literals), "Maximum"
    elif typeID == 5:
        return int(literals[0] > literals[1]), "Greater Than"
    elif typeID == 6:
        return int(literals[0] < literals[1]), "Less Than"
    elif typeID == 7:
        return int(literals[0] == literals[1]), "Equal To"
    else:
        "Error!!!"
        return 99999

In [159]:
def decode_hex(hex_str, verbose = False, veryverbose = False):
    version_sum = 0
    stream = get_binary(hex_str)
    packets = []
    i = 0
    pId = 1
    
    unparsedParents = []
    
    while (i < len(stream)):
        if veryverbose:
            print("\n\n====START OF NEW ITERATION====")
            print(stream[i:])
            print("ID number of new packet:", pId)
            
        subpacketComplete = False
        addedParentThisIteration = False
        i_start = i
        
        parentID = unparsedParents[-1]["ID"] if unparsedParents else "N/A - Outer Packet"
        
        #the first three bits encode the packet version
        version = stream[i:i+3]
        version = int(version, 2)
        if veryverbose:
            print("Packet Version:", version)
        version_sum += version

        #the next three bits encode the packet type ID
        typeid = stream[i+3:i+6]
        typeid = int(typeid, 2)
        if veryverbose:
            print("Packet Type ID:", typeid)
        
        packets.append({"ID":pId, "Version": version, "TypeID": typeid, "ParentID": parentID})
        i+=6
        
        if typeid == 4:
            litvalue, literal_end_index = evaluate_literal(stream[i:])
            i += literal_end_index
            if veryverbose:
                print(f"packet #{pId} encodes a literal value", litvalue)
            packets[-1]["LiteralValue"] = litvalue
            if unparsedParents:
                unparsedParents[-1]["Literals"].append(litvalue)
            subpacketComplete = True
            

        # Every other type of packet (any packet with a type ID other than 4) represent an operator packet
        else: 
            lengthTypeID = stream[i]
            i+=1
            # If the length type ID is 0, 
            # then the next 15 bits are a number that represents the total length in bits 
            # of the sub-packets contained by this packet.

            if int(lengthTypeID) == 0:
                lengthSubpackets = int(stream[i:i+15],2) # bits
                i+=15
                packets[-1]["Bits in Subpackets"]=lengthSubpackets
                unparsedParents.append({"ID": pId, "typeID": typeid, "type": 0, "Literals": [],
                                        "length": lengthSubpackets, "lengthParsed": 0, "done": False})
                if veryverbose:
                    print(f"packet #{pId} is an operator packet with {lengthSubpackets} bits in its subpackets")

            # If the length type ID is 1, 
            # then the next 11 bits are a number that 
            # represents the number of sub-packets immediately contained by this packet.
            else:
                numSubpackets = int(stream[i:i+11], 2) #subpackets
                i+=11
                packets[-1]["numSubpackets"]=numSubpackets
                unparsedParents.append({"ID": pId, "typeID": typeid, "type": 1, "Literals": [],
                                        "numSubpackets": numSubpackets, "packetsParsed": 0, "done": False})
                if veryverbose:
                    print(f"packet #{pId} is an operator packet with {numSubpackets} subpackets.")
                
            addedParentThisIteration = True
                
                
        bitsParsedThisIteration = i-i_start
        if veryverbose:
            print("Bits Parsed this Iteration:", bitsParsedThisIteration, stream[i_start:i])
        
        
        for parent in reversed(unparsedParents):
            if parent["type"] == 0:
                parent["lengthParsed"] += 0 if addedParentThisIteration else bitsParsedThisIteration
                if parent["lengthParsed"] >= parent["length"]:
                    parent["done"] = True
            addedParentThisIteration = False #reset, this is only relevant to innermost packet (the one added to stack last)
        
        while (unparsedParents):
            if subpacketComplete:
                if unparsedParents[-1]["type"] == 1:
                    unparsedParents[-1]["packetsParsed"] += 1
                    if unparsedParents[-1]["packetsParsed"] == unparsedParents[-1]["numSubpackets"]:
                        unparsedParents[-1]["done"] = True
                    else:
                        subpacketComplete = False
    
            if unparsedParents[-1]["done"]:
                finishedParent = unparsedParents.pop()
                packet = [x for x in packets if x["ID"] == finishedParent["ID"]][0]
                value, operator = evaluate_operator(finishedParent["typeID"], finishedParent["Literals"])
                packet["Value"] = value
                packet["Operator"] = operator
                packet["Literals"] = finishedParent["Literals"]
                
                if unparsedParents:
                    unparsedParents[-1]["Literals"].append(value)
                if veryverbose:
                    print(f"Packet ID #{finishedParent['ID']} is complete! This encodes a value of {value}")
            else:
                break
                
        if veryverbose:
            print('\nUNPARSED PARENTS')
            print(unparsedParents)
            print('\n')

            print('PACKETS')
            print(packets)
        
        pId+=1 #next packet ID
        
        if noSigBitsRemaining(stream, i):
            subpacketComplete = True
            i = len(stream)
            part2value = value
            if veryverbose:
                print("The remaining bits are 0s")
                print(f"The decoding has completed. The final value is {value}")
        
    if verbose:
        print(packets)
        
    return version_sum, part2value

In [90]:
puzzle_input = parse_data("InputFiles/day16input.txt")

In [161]:
decode_hex(puzzle_input)

(974, 180616437720)

In [160]:
decode_hex("9C0141080250320F1802104A08", verbose = True)

[{'ID': 1, 'Version': 4, 'TypeID': 7, 'ParentID': 'N/A - Outer Packet', 'Bits in Subpackets': 80, 'Value': 1, 'Operator': 'Equal To', 'Literals': [4, 4]}, {'ID': 2, 'Version': 2, 'TypeID': 0, 'ParentID': 1, 'numSubpackets': 2, 'Value': 4, 'Operator': 'Sum', 'Literals': [1, 3]}, {'ID': 3, 'Version': 2, 'TypeID': 4, 'ParentID': 2, 'LiteralValue': 1}, {'ID': 4, 'Version': 4, 'TypeID': 4, 'ParentID': 2, 'LiteralValue': 3}, {'ID': 5, 'Version': 6, 'TypeID': 1, 'ParentID': 1, 'numSubpackets': 2, 'Value': 4, 'Operator': 'Product', 'Literals': [2, 2]}, {'ID': 6, 'Version': 0, 'TypeID': 4, 'ParentID': 5, 'LiteralValue': 2}, {'ID': 7, 'Version': 2, 'TypeID': 4, 'ParentID': 5, 'LiteralValue': 2}]


(20, 1)